### 타이타닉 데이터 사용 예제(실습)
* targer: Servivied
* Passengerid, Name, Ticket, Cabin 컬럼은 삭제
* Nan 데이터는 삭제
* Sex, Emabrked는 레이블 인코딩(실제 값에 상관없이 0~k-1 까지의 정수로 변환하는 것)
* Age, Fare는 minmaxscaler로 전처리

전체 독립변수를 사용햇을때와 vif 10이상을 제거한 독립변수를 사용했을때의 ols 결과값 구하기(summary)

In [3]:
import pandas as pd

df = pd.read_csv('./titanic/train.csv')
df.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [21]:
df.dropna(inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
885,0,3,0,39.0,0,5,29.1250,1
886,0,2,1,27.0,0,0,13.0000,2
887,1,1,0,19.0,0,0,30.0000,2
889,1,1,1,26.0,0,0,30.0000,0


In [14]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [29]:
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

y_s = df.Sex
y_e = df.Embarked

le_s = LabelEncoder()
le_e = LabelEncoder()
# 0 ~ k-1까지 정수로 변환 작업

le_s.fit(y_s)
le_e.fit(y_e)
# 인코더 모델 생성 (학습)

df['Sex'] = le_s.transform(y_s)
df['Embarked'] = le_e.transform(y_e)
# 인코딩된 모델로 변환

df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,0.271174,1,0,0.014151,2
1,1,1,0,0.472229,1,0,0.139136,0
2,1,3,0,0.321438,0,0,0.015469,2
3,1,1,0,0.434531,1,0,0.103644,2
4,0,3,1,0.434531,0,0,0.015713,2
...,...,...,...,...,...,...,...,...
885,0,3,0,0.484795,0,5,0.056848,1
886,0,2,1,0.334004,0,0,0.025374,2
887,1,1,0,0.233476,0,0,0.058556,2
889,1,1,1,0.321438,0,0,0.058556,0


In [30]:
from sklearn.preprocessing import MinMaxScaler
# Age, Fare는 minmaxscaler로 전처리

sc = MinMaxScaler()

x = df[['Age']]
x_scaled = sc.fit_transform(x)

x1 = df[['Fare']]
x1_scaled = sc.fit_transform(x1)

df[['Age']] = x_scaled
df[['Fare']] = x1_scaled

df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,0.271174,1,0,0.014151,2
1,1,1,0,0.472229,1,0,0.139136,0
2,1,3,0,0.321438,0,0,0.015469,2
3,1,1,0,0.434531,1,0,0.103644,2
4,0,3,1,0.434531,0,0,0.015713,2
...,...,...,...,...,...,...,...,...
885,0,3,0,0.484795,0,5,0.056848,1
886,0,2,1,0.334004,0,0,0.025374,2
887,1,1,0,0.233476,0,0,0.058556,2
889,1,1,1,0.321438,0,0,0.058556,0


In [33]:
formula = 'Survived~' + '+'.join(df.iloc[:, 1:].columns)
formula

'Survived~Pclass+Sex+Age+SibSp+Parch+Fare+Embarked'

In [36]:
from patsy import dmatrices

y, X = dmatrices(formula, df, return_type='dataframe')

y, X

(     Survived
 0         0.0
 1         1.0
 2         1.0
 3         1.0
 4         0.0
 ..        ...
 885       0.0
 886       0.0
 887       1.0
 889       1.0
 890       0.0
 
 [712 rows x 1 columns],
      Intercept  Pclass  Sex       Age  SibSp  Parch      Fare  Embarked
 0          1.0     3.0  1.0  0.271174    1.0    0.0  0.014151       2.0
 1          1.0     1.0  0.0  0.472229    1.0    0.0  0.139136       0.0
 2          1.0     3.0  0.0  0.321438    0.0    0.0  0.015469       2.0
 3          1.0     1.0  0.0  0.434531    1.0    0.0  0.103644       2.0
 4          1.0     3.0  1.0  0.434531    0.0    0.0  0.015713       2.0
 ..         ...     ...  ...       ...    ...    ...       ...       ...
 885        1.0     3.0  0.0  0.484795    0.0    5.0  0.056848       1.0
 886        1.0     2.0  1.0  0.334004    0.0    0.0  0.025374       2.0
 887        1.0     1.0  0.0  0.233476    0.0    0.0  0.058556       2.0
 889        1.0     1.0  1.0  0.321438    0.0    0.0  0.058556 

In [37]:
import pandas as pd

vif = pd.DataFrame()

In [43]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif['VIF Factor'] = [variance_inflation_factor(X.values, i)
                     for i in range(X.shape[1])]

# func(a, i) for i range(x[1])

vif['Features'] = X.columns

vif

,VIF Factor,Features
0,32.134512,Intercept
1,1.749120,Pclass
2,1.119651,Sex
3,1.310796,Age
4,1.276469,SibSp
5,1.288619,Parch
6,1.641348,Fare
7,1.114880,Embarked


In [62]:
def get_vif(formula, df):
    from patsy import dmatrices
    import pandas as pd
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    
    y, X = dmatrices(formula, df, return_type='dataframe')
    
    vif = pd.DataFrame()
    
    vif['VIF Factor'] = [variance_inflation_factor(X.values, i)
                         for i in range(X.shape[1])]
    
    vif['Features'] = X.columns
    
    return vif

formula = 'Survived~' + '+'.join(df.iloc[:, 1:].columns) + '-1'

get_vif(formula, df)

,VIF Factor,Features
0,5.974701,Pclass
1,3.034949,Sex
2,4.104924,Age
3,1.639080,SibSp
4,1.618650,Parch
5,1.655179,Fare
6,5.278139,Embarked


In [63]:
formula

'Survived~Pclass+Sex+Age+SibSp+Parch+Fare+Embarked-1'

In [65]:
import statsmodels.formula.api as smf

model_titanic = smf.ols(formula=formula, data=df).fit()
model_titanic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               Survived   R-squared (uncentered):                   0.492
Model:                            OLS   Adj. R-squared (uncentered):              0.487
Method:                 Least Squares   F-statistic:                              97.61
Date:                Sat, 08 Oct 2022   Prob (F-statistic):                    2.24e-99
Time:                        13:40:55   Log-Likelihood:                         -446.83
No. Observations:                 712   AIC:                                      907.7
Df Residuals:                     705   BIC:                                      939.6
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass         0.1077      0.017      6.170      0.000       0.073       0.142
Sex           -0.4200      0.037    -11.264      0.000      -0.493      -0.347
Age            0.4450      0.084      5.274      0.000       0.279       0.611
SibSp         -0.0135      0.021     -0.655      0.513      -0.054       0.027
Parch         -0.0023      0.023     -0.102      0.919      -0.047       0.042
Fare           1.7641      0.178      9.908      0.000       1.415       2.114
Embarked       0.0705      0.022      3.190      0.001       0.027       0.114
==============================================================================
Omnibus:                       53.916   Durbin-Watson:                   1.855
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.943
Skew:                           0.430   Prob(JB):                     1.57e-08
Kurtosis:                       2.313   Cond. No.                         32.8
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""